In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np 
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [43]:
n=pd.read_csv("./chronic_kidney_with_class.csv")

In [61]:
n.head()

,Unnamed: 0,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,0,48,80,1.020,1,0,NaN,1.0,0.0,0.0,...,44,7800,5.2,1.0,1,0.0,1.0,0.0,0.0,1
1,1,7,50,1.020,4,0,NaN,1.0,0.0,0.0,...,38,6000,NaN,0.0,0,0.0,1.0,0.0,0.0,1
2,2,62,80,1.010,2,3,1.0,1.0,0.0,0.0,...,31,7500,NaN,0.0,1,0.0,0.0,0.0,1.0,1
3,3,48,70,1.005,4,0,1.0,0.0,1.0,0.0,...,32,6700,3.9,1.0,0,0.0,0.0,1.0,1.0,1
4,4,51,80,1.010,2,0,1.0,1.0,0.0,0.0,...,35,7300,4.6,0.0,0,0.0,1.0,0.0,0.0,1


In [6]:
n.columns

Index(['Unnamed: 0', 'age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod',
       'pot', 'hemo', 'pcv', 'wc', 'rc', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm',
       'cad', 'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [18]:
n.shape

(400, 26)

In [45]:
n.replace("?",np.nan,inplace=True)
n.head()

,Unnamed: 0,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,0,48,80,1.020,1,0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7,50,1.020,4,0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62,80,1.010,2,3,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48,70,1.005,4,0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51,80,1.010,2,0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [47]:
n[["pot","pcv","hemo","wc","rc","rbc"]]

,pot,pcv,hemo,wc,rc,rbc
0,121,1.2,36,NaN,NaN,15.4
1,NaN,0.8,18,NaN,NaN,11.3
2,423,1.8,53,NaN,NaN,9.6
3,117,3.8,56,111,2.5,11.2
4,106,1.4,26,NaN,NaN,11.6
5,74,1.1,25,142,3.2,12.2
6,100,24.0,54,104,4.0,12.4
7,410,1.1,31,NaN,NaN,12.4
8,138,1.9,60,NaN,NaN,10.8
9,70,7.2,107,114,3.7,9.5


In [51]:
n.replace("yes",1,inplace=True)
n.replace("no",0,inplace=True)
n.replace("normal",1,inplace=True)
n.replace("abnormal",0,inplace=True)
n.replace("present",1,inplace=True)
n.replace("notpresent",0,inplace=True)
n.replace("good",1,inplace=True)
n.replace("poor",0,inplace=True)
n.replace("ckd",1,inplace=True)
n.replace("notckd",0,inplace=True)
n.head()

,Unnamed: 0,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,0,48,80,1.020,1,0,NaN,1.0,0.0,0.0,...,44,7800,5.2,1.0,1,0.0,1.0,0.0,0.0,1
1,1,7,50,1.020,4,0,NaN,1.0,0.0,0.0,...,38,6000,NaN,0.0,0,0.0,1.0,0.0,0.0,1
2,2,62,80,1.010,2,3,1.0,1.0,0.0,0.0,...,31,7500,NaN,0.0,1,0.0,0.0,0.0,1.0,1
3,3,48,70,1.005,4,0,1.0,0.0,1.0,0.0,...,32,6700,3.9,1.0,0,0.0,0.0,1.0,1.0,1
4,4,51,80,1.010,2,0,1.0,1.0,0.0,0.0,...,35,7300,4.6,0.0,0,0.0,1.0,0.0,0.0,1


In [52]:
n.describe()

,Unnamed: 0,bgr,bu,sc,sod,htn,cad,appet,pe,ane,class
count,400.000000,248.000000,335.000000,396.000000,396.000000,398.000000,398.000000,399.000000,399.000000,399.000000,400.000000
mean,199.500000,0.810484,0.773134,0.106061,0.055556,0.369347,0.085427,0.794486,0.190476,0.150376,0.625000
std,115.614301,0.392711,0.419431,0.308305,0.229351,0.483235,0.279868,0.404584,0.393170,0.357888,0.484729
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,99.750000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,199.500000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,299.250000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,399.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [67]:
n1=n.dropna(axis=0)
n1.head()

,Unnamed: 0,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
3,3,48,70,1.005,4,0,1.0,0.0,1.0,0.0,...,32,6700,3.9,1.0,0,0.0,0.0,1.0,1.0,1
9,9,53,90,1.020,2,0,0.0,0.0,1.0,0.0,...,29,12100,3.7,1.0,1,0.0,0.0,0.0,1.0,1
11,11,63,70,1.010,3,0,0.0,0.0,1.0,0.0,...,32,4500,3.8,1.0,1,0.0,0.0,1.0,0.0,1
14,14,68,80,1.010,3,2,1.0,0.0,1.0,1.0,...,16,11000,2.6,1.0,1,1.0,0.0,1.0,0.0,1
20,20,61,80,1.015,2,0,0.0,0.0,0.0,0.0,...,24,9200,3.2,1.0,1,1.0,0.0,1.0,1.0,1


In [60]:
n1["class"].value_counts()

0    115
1     43
Name: class, dtype: int64

In [74]:
n1.drop("bp",axis=1,inplace=True)
print(n1)

        sg al su  bgr   bu   sc  sod  pot hemo   pcv  ...    pc    pcc   ba  \
3    1.005  4  0  1.0  0.0  1.0  0.0  117   56   3.8  ...    32   6700  3.9   
9    1.020  2  0  0.0  0.0  1.0  0.0   70  107   7.2  ...    29  12100  3.7   
11   1.010  3  0  0.0  0.0  1.0  0.0  380   60   2.7  ...    32   4500  3.8   
14   1.010  3  2  1.0  0.0  1.0  1.0  157   90   4.1  ...    16  11000  2.6   
20   1.015  2  0  0.0  0.0  0.0  0.0  173  148   3.9  ...    24   9200  3.2   
22   1.025  4  0  1.0  0.0  0.0  0.0   95  163   7.7  ...    32   6900  3.4   
27   1.010  3  4  1.0  0.0  0.0  0.0  264   87   2.7  ...    37   9600  4.1   
48   1.005  0  0  1.0  1.0  0.0  0.0   70   32   0.9  ...    29  18900  3.5   
58   1.020  2  0  0.0  0.0  0.0  0.0  253  142   4.6  ...    33   7200  4.3   
71   1.010  1  0  1.0  1.0  0.0  0.0  163   92   3.3  ...    28  14600  3.2   
74   1.015  2  0  0.0  0.0  0.0  0.0  129  107   6.7  ...    29   6400  3.4   
76   1.005  4  0  0.0  0.0  0.0  1.0  133  139   8.5

C:\Users\My Dell\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
